# Merging Dataframes in Pandas - Concat and Merge


In [1]:
import pandas as pd

### Concatenating using append

A useful shortcut to `concat()` are the `append()` instance methods on Series and DataFrame. These methods actually predated concat. They concatenate along axis=0, namely the index:

In [11]:
result = df1.append(df2)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In the case of DataFrame, the indexes must be disjoint but the columns do not need to be:

In [12]:
result = df1.append(df4, sort=False)
display(df1, df4, result)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
2,NaN,B2,NaN,D2,F2
3,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


**Note:** Unlike the `append()` method, which appends to the original list and returns `None`, `append()` here does __not modify__ df1 and instead returns its copy with df2 appended.

### Ignoring indexes on the concatenation axis

For DataFrame objects which don’t have a meaningful index, you may wish to append them and ignore the fact that they may have overlapping indexes. To do this, use the `ignore_index` argument:

In [13]:
result = pd.concat([df1, df4], ignore_index=True, sort=False)
result

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
4,NaN,B2,NaN,D2,F2
5,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


In [14]:
result = df1.append(df4, ignore_index=True, sort=False)
result

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
4,NaN,B2,NaN,D2,F2
5,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


### Concatenating with mixed ndims

In [15]:
s1 = pd.Series(['X0', 'X1', 'X2', 'X3'], name='X')
result = pd.concat([df1, s1], axis=1)
display(s1, df1, result)

0    X0
1    X1
2    X2
3    X3
Name: X, dtype: object

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


,A,B,C,D,X
0,A0,B0,C0,D0,X0
1,A1,B1,C1,D1,X1
2,A2,B2,C2,D2,X2
3,A3,B3,C3,D3,X3


**Note:** Since we’re concatenating a Series to a DataFrame, we could have achieved the same result with `DataFrame.assign()`. To concatenate an arbitrary number of pandas objects (DataFrame or Series), use `concat`.

In [16]:
df1.assign(X = s1)

,A,B,C,D,X
0,A0,B0,C0,D0,X0
1,A1,B1,C1,D1,X1
2,A2,B2,C2,D2,X2
3,A3,B3,C3,D3,X3


In [17]:
# If unnamed Series are passed they will be numbered consecutively.
s2 = pd.Series(['_0', '_1', '_2', '_3'])
result = pd.concat([df1, s2, s2, s2], axis=1)
result

,A,B,C,D,0,1,2
0,A0,B0,C0,D0,_0,_0,_0
1,A1,B1,C1,D1,_1,_1,_1
2,A2,B2,C2,D2,_2,_2,_2
3,A3,B3,C3,D3,_3,_3,_3


### Using `assign()`

``
DataFrame.assign(self, **kwargs)
``

* Assign new columns to a DataFrame.
* Returns a new object with all original columns in addition to new ones. Existing columns that are re-assigned will be overwritten.
* Parameters:
    - `self`
    - `**kwargs`: dict of {str: callable or Series}. The column names are keywords. If the values are callable, they are computed on the DataFrame and assigned to the new columns. The callable must not change input DataFrame (though pandas doesn’t check it). If the values are not callable, (e.g. a Series, scalar, or array), they are simply assigned.

In [18]:
df = pd.DataFrame({'temp_c': [17.0, 25.0]},
                  index=['Portland', 'Berkeley'])
df

,temp_c
Portland,17.0
Berkeley,25.0


In [19]:
# Where the value is a callable, evaluated on df:
df.assign(temp_f=lambda x: x.temp_c * 9 / 5 + 32)

,temp_c,temp_f
Portland,17.0,62.6
Berkeley,25.0,77.0


In [20]:
# Alternatively, the same behavior can be achieved by directly referencing an existing Series or sequence:
df.assign(temp_f=df['temp_c'] * 9 / 5 + 32)

,temp_c,temp_f
Portland,17.0,62.6
Berkeley,25.0,77.0


In [21]:
# You can create multiple columns within the same assign 
# where one of the columns depends on another one defined 
# within the same assign:
df.assign(temp_f=lambda x: x['temp_c'] * 9 / 5 + 32,
          temp_k=lambda x: (x['temp_f'] +  459.67) * 5 / 9)

,temp_c,temp_f,temp_k
Portland,17.0,62.6,290.15
Berkeley,25.0,77.0,298.15


### More concatenating with group keys

* A fairly common use of the `keys` argument is to override the column names when creating a new DataFrame based on existing Series
* Notice how the default behaviour consists on letting the resulting DataFrame inherit the parent Series’ name, when these existed.

In [22]:
s3 = pd.Series([0, 1, 2, 3], name='foo')
s4 = pd.Series([0, 1, 2, 3])
s5 = pd.Series([0, 1, 4, 5])
pd.concat([s3, s4, s5], axis=1)

,foo,0,1
0,0,0,0
1,1,1,1
2,2,2,4
3,3,3,5


In [23]:
# Through the keys argument we can override the existing column names.
pd.concat([s3, s4, s5], axis=1, keys=['red', 'blue', 'yellow'])

,red,blue,yellow
0,0,0,0
1,1,1,1
2,2,2,4
3,3,3,5


In [24]:
# Let’s consider a variation of the very first example presented:
result = pd.concat(frames, keys=['x', 'y', 'z'])
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

In [25]:
# You can also pass a dict to concat in which case the dict keys will be used for the keys argument
pieces = {'x': df1, 'y': df2, 'z': df3}
result = pd.concat(pieces)
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

In [26]:
# MultiIndex created has levels that are constructed from the passed keys and the index of the DataFrame pieces:
result.index.levels

FrozenList([['x', 'y', 'z'], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]])

In [27]:
# If you wish to specify other levels (as will occasionally be the case), you can do so using the levels argument:
result = pd.concat(pieces, keys=['x', 'y', 'z'],
                           levels=[['z', 'y', 'x', 'w']],
                           names=['group_key'])
result

A    B    C    D
group_key                       
x         0    A0   B0   C0   D0
          1    A1   B1   C1   D1
          2    A2   B2   C2   D2
          3    A3   B3   C3   D3
y         4    A4   B4   C4   D4
          5    A5   B5   C5   D5
          6    A6   B6   C6   D6
          7    A7   B7   C7   D7
z         8    A8   B8   C8   D8
          9    A9   B9   C9   D9
          10  A10  B10  C10  D10
          11  A11  B11  C11  D11

In [28]:
result.index.levels

FrozenList([['z', 'y', 'x', 'w'], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]])

### Appending rows to a DataFrame

* use `ignore_index` with this method to instruct DataFrame to discard its index

In [29]:
dicts = [{'A': 1, 'B': 2, 'C': 3, 'X': 4},
         {'A': 5, 'B': 6, 'C': 7, 'Y': 8}]
result = df1.append(dicts, ignore_index=True, sort=False)
display(dicts, df1, result)

[{'A': 1, 'B': 2, 'C': 3, 'X': 4}, {'A': 5, 'B': 6, 'C': 7, 'Y': 8}]

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


,A,B,C,D,X,Y
0,A0,B0,C0,D0,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN
4,1,2,3,NaN,4.0,NaN
5,5,6,7,NaN,NaN,8.0


## Concatenating many files

### Globbing

* Pattern matching for file names
* `glob()` function to find files based on pattern
* Wildcards: `*` and `?`
    - Any csv file: `*.csv`
    - Any single character: `file_?.csv`
* Returns list of file names

In [30]:
import glob

pattern = '*.csv'
csv_files = glob.glob('data/' + pattern)
csv_files

['data/sp500.csv',
 'data/all_medalists.csv',
 'data/pittsburgh2013.csv',
 'data/pennsylvania2012_turnout.csv',
 'data/titanic.csv',
 'data/exchange.csv',
 'data/gapminder_tidy.csv',
 'data/users.csv']

### Iterating and concatenating all matches

1. iterate through each of the filenames
2. read each filename into a DataFrame, and then
3. append it to the `frames` list.

In [31]:
# Create an empty list: frames
frames = []

#  Iterate over csv_files
for csv in csv_files:
    
    #  Read csv into a DataFrame: df
    df = pd.read_csv(csv)
    # Append df to frames
    frames.append(df)
    
# Concatenate frames into a single DataFrame: my_files
my_files = pd.concat(frames)

print(my_files.shape)
print(my_files.head())

(41574, 76)
         Date         Open         High          Low        Close  \
0  2015-01-02  2058.899902  2072.360107  2046.040039  2058.199951   
1  2015-01-05  2054.439941  2054.439941  2017.339966  2020.579956   
2  2015-01-06  2022.150024  2030.250000  1992.439941  2002.609985   
3  2015-01-07  2005.550049  2029.609985  2005.550049  2025.900024   
4  2015-01-08  2030.609985  2064.080078  2030.609985  2062.139893   

         Volume    Adj Close City  Edition Sport  ... life population  \
0  2.708700e+09  2058.199951  NaN      NaN   NaN  ...  NaN        NaN   
1  3.799120e+09  2020.579956  NaN      NaN   NaN  ...  NaN        NaN   
2  4.460110e+09  2002.609985  NaN      NaN   NaN  ...  NaN        NaN   
3  3.805480e+09  2025.900024  NaN      NaN   NaN  ...  NaN        NaN   
4  3.934010e+09  2062.139893  NaN      NaN   NaN  ...  NaN        NaN   

  child_mortality gdp region Unnamed: 0 weekday  city  visitors  signups  
0             NaN NaN    NaN        NaN     NaN   NaN      

## Merging data

```
pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
```

* `left`: A DataFrame or named Series object.
* `right`: Another DataFrame or named Series object.
* `on`: Column or index level names to join on. Must be found in both the left and right DataFrame and/or Series objects. If not passed and left_index and right_index are False, the intersection of the columns in the DataFrames and/or Series will be inferred to be the join keys.
* `left_on`: Columns or index levels from the left DataFrame or Series to use as keys. Can either be column names, index level names, or arrays with length equal to the length of the DataFrame or Series.
* `right_on`: Columns or index levels from the right DataFrame or Series to use as keys. Can either be column names, index level names, or arrays with length equal to the length of the DataFrame or Series.
* `left_index`: If True, use the index (row labels) from the left DataFrame or Series as its join key(s). In the case of a DataFrame or Series with a MultiIndex (hierarchical), the number of levels must match the number of join keys from the right DataFrame or Series.
* `right_index`: Same usage as left_index for the right DataFrame or Series
* `how`: One of 'left', 'right', 'outer', 'inner'. Defaults to inner. See below for more detailed description of each method.
* `sort`: Sort the result DataFrame by the join keys in lexicographical order. Defaults to True, setting to False will improve performance substantially in many cases.
* `suffixes`: A tuple of string suffixes to apply to overlapping columns. Defaults to ('_x', '_y').
* `copy`: Always copy data (default True) from the passed DataFrame or named Series objects, even when reindexing is not necessary. Cannot be avoided in many cases but may improve performance / memory usage. The cases where copying can be avoided are somewhat pathological but this option is provided nonetheless.
* `indicator`: Add a column to the output DataFrame called _merge with information on the source of each row. _merge is Categorical-type and takes on a value of left_only for observations whose merge key only appears in 'left' DataFrame or Series, right_only for observations whose merge key only appears in 'right' DataFrame or Series, and both if the observation’s merge key is found in both.
* `validate`: string, default None. If specified, checks if merge is of specified type.
    * “one_to_one” or “1:1”: checks if merge keys are unique in both left and right datasets.
    * “one_to_many” or “1:m”: checks if merge keys are unique in left dataset.
    * “many_to_one” or “m:1”: checks if merge keys are unique in right dataset.
    * “many_to_many” or “m:m”: allowed, but does not result in checks.
    
### Note
* The return type will be the same as `left`. If left is a DataFrame or named Series and right is a subclass of DataFrame, the return type will still be DataFrame.
* `join()` method, uses merge internally for the index-on-index (by default) and column(s)-on-index join. If you are joining on index only, you may wish to use `DataFrame.join` to save yourself some typing.

In [32]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

result = pd.merge(left, right, on='key')
result

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


* Here is a more complicated example with multiple join keys. Only the keys appearing in left and right are present (the intersection), since `how='inner'` by default.

In [33]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
display(left, right)

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


In [34]:
result = pd.merge(left, right, on=['key1', 'key2'])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


* If a key combination does not appear in either the left or right tables, the values in the joined table will be `NA`
* Here is a summary of the how options and their SQL equivalent names:

| Merge method | SQL Join Name    | Description                               |
|--------------|------------------|-------------------------------------------|
| left         | LEFT OUTER JOIN  | Use keys from left frame only             |
| right        | RIGHT OUTER JOIN | Use keys from right frame only            |
| outer        | FULL OUTER JOIN  | Use union of keys from both frames        |
| inner        | INNER JOIN       | Use intersection of keys from both frames |

In [35]:
result = pd.merge(left, right, how='left', on=['key1', 'key2'])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN


In [36]:
result = pd.merge(left, right, how='right', on=['key1', 'key2'])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


In [37]:
result = pd.merge(left, right, how='outer', on=['key1', 'key2'])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


In [38]:
result = pd.merge(left, right, how='inner', on=['key1', 'key2'])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


In [39]:
# with duplicate key joins
left = pd.DataFrame({'A': [1, 2], 'B': [2, 2]})
right = pd.DataFrame({'A': [4, 5, 6], 'B': [2, 2, 2]})
result = pd.merge(left, right, on='B', how='outer')
display(left, right, result)

,A,B
0,1,2
1,2,2


,A,B
0,4,2
1,5,2
2,6,2


,A_x,B,A_y
0,1,2,4
1,1,2,5
2,1,2,6
3,2,2,4
4,2,2,5
5,2,2,6


### Checking for duplicate keys

* Users can use the `validate` argument to automatically check whether there are unexpected duplicates in their merge keys - exception is raised otherwise
* Key uniqueness is checked before merge operations and so should protect against memory overflows.

In [40]:
left = pd.DataFrame({'A' : [1,2], 'B' : [1, 2]})
right = pd.DataFrame({'A' : [4,5,6], 'B': [2, 2, 2]})
result = pd.merge(left, right, on='B', how='outer', validate="one_to_one")

MergeError: Merge keys are not unique in right dataset; not a one-to-one merge

* If the user is aware of the duplicates in the right DataFrame but wants to ensure there are no duplicates in the left DataFrame, one can use the `validate='one_to_many'` argument instead, which will not raise an exception.

In [41]:
pd.merge(left, right, on='B', how='outer', validate="one_to_many")

,A_x,B,A_y
0,1,1,NaN
1,2,2,4.0
2,2,2,5.0
3,2,2,6.0


### The merge indicator

* `merge()` accepts the argument `indicator`. If True, a Categorical-type column called `_merge` will be added to the output object that takes on values:

| Observation Origin              | `_merge` value |
|---------------------------------|----------------|
| Merge key only in 'left' frame  | left_only      |
| Merge key only in 'right' frame | right_only     |
| Merge key in both frames        | both           |

In [42]:
df1 = pd.DataFrame({'col1': [0, 1], 'col_left': ['a', 'b']})
df2 = pd.DataFrame({'col1': [1, 2, 2], 'col_right': [2, 2, 2]})
pd.merge(df1, df2, on='col1', how='outer', indicator=True)

,col1,col_left,col_right,_merge
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


In [43]:
pd.merge(df1, df2, on='col1', how='inner', indicator=True)

,col1,col_left,col_right,_merge
0,1,b,2,both


In [44]:
pd.merge(df1, df2, on='col1', how='left', indicator=True)

,col1,col_left,col_right,_merge
0,0,a,NaN,left_only
1,1,b,2.0,both


In [45]:
pd.merge(df1, df2, on='col1', how='right', indicator=True)

,col1,col_left,col_right,_merge
0,1,b,2,both
1,2,NaN,2,right_only
2,2,NaN,2,right_only


In [46]:
# also accepts string arguments
pd.merge(df1, df2, on='col1', how='outer', indicator='indicator_column')

,col1,col_left,col_right,indicator_column
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


## `pandas.merge_ordered()`

```python
pandas.merge_ordered(left, right, on=None, left_on=None, right_on=None, left_by=None, right_by=None, fill_method=None, suffixes='_x', '_y', how: str = 'outer') → ’DataFrame’
```

In [48]:
A = pd.read_clipboard()

In [57]:
A

,key,lvalue,group
0,a,1,a
1,c,2,a
2,e,3,a
3,a,1,b
4,c,2,b
5,e,3,b


In [54]:
B = pd.read_clipboard()

In [58]:
B

,Key,rvalue
0,b,1
1,c,2
2,d,3


In [60]:
pd.merge_ordered(A, B, fill_method='ffill', left_by='group', left_on = ['key'], right_on = ['Key'])

,key,lvalue,group,Key,rvalue
0,a,1,a,NaN,NaN
1,a,1,a,b,1.0
2,c,2,a,c,2.0
3,c,2,a,d,3.0
4,e,3,a,d,3.0
5,a,1,b,NaN,NaN
6,a,1,b,b,1.0
7,c,2,b,c,2.0
8,c,2,b,d,3.0
9,e,3,b,d,3.0


In [61]:
pd.merge_ordered(A, B, left_on = ['key'], right_on = ['Key'])

,key,lvalue,group,Key,rvalue
0,a,1.0,a,NaN,NaN
1,a,1.0,b,NaN,NaN
2,NaN,NaN,NaN,b,1.0
3,c,2.0,a,c,2.0
4,c,2.0,b,c,2.0
5,NaN,NaN,NaN,d,3.0
6,e,3.0,a,NaN,NaN
7,e,3.0,b,NaN,NaN


In [62]:
pd.merge_ordered(A, B, left_by='group', left_on = ['key'], right_on = ['Key'])

,key,lvalue,group,Key,rvalue
0,a,1.0,a,NaN,NaN
1,NaN,NaN,a,b,1.0
2,c,2.0,a,c,2.0
3,NaN,NaN,a,d,3.0
4,e,3.0,a,NaN,NaN
5,a,1.0,b,NaN,NaN
6,NaN,NaN,b,b,1.0
7,c,2.0,b,c,2.0
8,NaN,NaN,b,d,3.0
9,e,3.0,b,NaN,NaN


In [67]:
print(A,'\n', B)

  key  lvalue group
0   a       1     a
1   c       2     a
2   e       3     a
3   a       1     b
4   c       2     b
5   e       3     b 
   Key  rvalue
0   b       1
1   c       2
2   d       3


## pandas.merge_asof()

```python
pandas.merge_asof(left, right, on=None, left_on=None, right_on=None, left_index: bool = False, right_index: bool = False, by=None, left_by=None, right_by=None, suffixes='_x', '_y', tolerance=None, allow_exact_matches: bool = True, direction: str = 'backward') → ’DataFrame’
```

Perform an asof merge. This is similar to a left-join except that we match on nearest key rather than equal keys.

Both DataFrames must be sorted by the key.

For each row in the left DataFrame:
* A “backward” search selects the last row in the right DataFrame whose ‘on’ key is less than or equal to the left’s key.
* A “forward” search selects the first row in the right DataFrame whose ‘on’ key is greater than or equal to the left’s key.
* A “nearest” search selects the row in the right DataFrame whose ‘on’ key is closest in absolute distance to the left’s key.

In [2]:
left = pd.DataFrame({'a': [1, 5, 10], 'left_val': ['a', 'b', 'c']})

In [3]:
right = pd.DataFrame({'a': [1, 2, 3, 6, 7],
                      'right_val': [1, 2, 3, 6, 7]})

In [5]:
left

,a,left_val
0,1,a
1,5,b
2,10,c


In [6]:
right

,a,right_val
0,1,1
1,2,2
2,3,3
3,6,6
4,7,7


In [7]:
pd.merge_asof(left, right, on='a') # left > right

,a,left_val,right_val
0,1,a,1
1,5,b,3
2,10,c,7


In [8]:
pd.merge_asof(left, right, on='a', allow_exact_matches=False)

,a,left_val,right_val
0,1,a,NaN
1,5,b,3.0
2,10,c,7.0


allow_exact_matches: bool, default True
* If True, allow matching with the same ‘on’ value (i.e. less-than-or-equal-to / greater-than-or-equal-to)
* If False, don’t match the same ‘on’ value (i.e., strictly less-than / strictly greater-than).

In [9]:
pd.merge_asof(left, right, on='a', direction='forward')

,a,left_val,right_val
0,1,a,1.0
1,5,b,6.0
2,10,c,NaN


In [10]:
pd.merge_asof(left, right, on='a', direction='nearest')

,a,left_val,right_val
0,1,a,1
1,5,b,6
2,10,c,7


Here is a real-world times-series example

In [35]:
quotes = pd.read_clipboard()

In [36]:
quotes.reset_index(inplace = True)

In [37]:
quotes['time'] = quotes['level_1'] + ' ' + quotes['time']

In [38]:
quotes.drop(['level_0','level_1'], axis = 1, inplace = True)

In [39]:
trades = pd.read_clipboard()

In [40]:
trades.reset_index(inplace = True)

In [41]:
trades['time'] = trades['level_1'] + ' ' + trades['time']

In [42]:
trades.drop(['level_0','level_1'], axis = 1, inplace = True)

In [44]:
trades['time'] = pd.to_datetime(trades['time'])
quotes['time'] = pd.to_datetime(quotes['time'])

We only asof within 2ms between the quote time and the trade time

In [45]:
pd.merge_asof(trades, quotes,
                      on='time',
                      by='ticker',
                      tolerance=pd.Timedelta('2ms'))

,time,ticker,price,quantity,bid,ask
0,2016-05-25 13:30:00.023,MSFT,51.95,75,51.95,51.96
1,2016-05-25 13:30:00.038,MSFT,51.95,155,NaN,NaN
2,2016-05-25 13:30:00.048,GOOG,720.77,100,720.50,720.93
3,2016-05-25 13:30:00.048,GOOG,720.92,100,720.50,720.93
4,2016-05-25 13:30:00.048,AAPL,98.00,100,NaN,NaN
